## Imports

In [93]:
import json
import re
from matplotlib import rcParams
rcParams['figure.figsize'] = 11.7,8.27
import nltk
from string import punctuation
import numpy as np


## Loading Data

In [94]:
def load_data_from_json(filename):
    with open(filename,'r') as f:
        return json.load(f)

In [95]:
master_data = load_data_from_json('dataset/master.json')

In [96]:
# master_data

In [97]:
def is_post(s):
    if len(re.findall(r'\$([a-zA-Z_]+)',s))>0:
        return True
    return False

In [98]:
#seprating post and headlines
post_sentence = []
headlines_sentence = []
for key in master_data.keys():
    if is_post(master_data[key]['sentence']):
        post_sentence.append(master_data[key]['sentence'])
    else:
        headlines_sentence.append(master_data[key]['sentence'])

In [99]:
len(post_sentence),len(headlines_sentence)

(675, 438)

In [88]:
# turn a sentence into clean tokens
def clean_sentence(sentence):
    #remove multiple repeat non num-aplha char !!!!!!!!!-->!
    sentence = re.sub(r'(\W)\1{2,}', r'\1', sentence) 
    #removes alpha char repeating more than twice aaaa->aa
    sentence = re.sub(r'(\w)\1{2,}', r'\1\1', sentence)
    #removes links
    sentence = re.sub(r'(?P<url>https?://[^\s]+)', r'', sentence)
    # remove @usernames
    sentence = re.sub(r"\@(\w+)", "", sentence)
    #removing stock names to see if it helps
#     sentence = re.sub(r"(?:\$|https?\://)\S+", "", sentence)
    #remove # from #tags
    sentence = sentence.replace('#','')
    # split into tokens by white space
    tokens = sentence.split()
    # remove punctuation from each token
    table = str.maketrans('', '', punctuation.replace('$',''))
    tokens = [w.translate(table) for w in tokens]
#     remove remaining tokens that are not alphabetic
#     tokens = [word for word in tokens if word.isalpha()]
#no removing non alpha words to keep stock names($ZSL)
    # filter out stop words
    stop_words = set(nltk.corpus.stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    tokens = ' '.join(tokens)
    return tokens

In [100]:
post_sentence = [clean_sentence(s) for s in post_sentence]
headlines_sentence = [clean_sentence(s) for s in headlines_sentence]

In [122]:
def prepare_sentence(s):
    sentences = nltk.sent_tokenize(s)
    tokenized_sentences = [nltk.word_tokenize(sentence) for sentence in sentences]
    tagged_sentences = [nltk.pos_tag(sentence) for sentence in tokenized_sentences]
    chunked_sentences = nltk.ne_chunk_sents(tagged_sentences, binary=True)
    return chunked_sentences

In [121]:
def extract_entity_names(t):
    entity_names = []

    if hasattr(t, 'label') and t.label:
        if t.label() == 'NE':
            entity_names.append(' '.join([child[0] for child in t]))
        else:
            for child in t:
                entity_names.extend(extract_entity_names(child))
    return entity_names

In [123]:
def get_ners(s):
    chunked_sentences = prepare_sentence(s)
    entity_names = []
    for tree in chunked_sentences:
        entity_names.extend(extract_entity_names(tree))
    entity_names = [for x in ]
    print(list(set(entity_names)))
#     return 

In [117]:
def get_index_of_targets(s,is_post_flag = True):
    if is_post_flag:
        targets = ['$'+x for x in re.findall(r'\$([a-zA-Z_]+)',s)]
        print(targets)
        index = [i for i,j in enumerate(s.split()) if j in targets]
        return index
    else:
        

In [107]:
def get_sentence_tokens_prob(s):
    s_prob = np.zeros(len(s.split()))
    print(s_prob)

In [125]:
get_ners('Royal Mail chairman Donald Brydon set step')

['Donald Brydon', 'Royal Mail']


In [124]:
headlines_sentence

['Royal Mail chairman Donald Brydon set step',
 'Stakes High AstraZeneca Heart Drug Facing Tough Competition',
 'UPDATE 1Dairy Crest loses third Morrisons milk contract',
 'Insight hires Avivas David Hillier multiasset team',
 'Primark racks happy Christmas strong sales',
 'UPDATE 1Pearson expects grow year solid end 2014',
 'Tesco sells Blinkbox broadband service TalkTalk',
 'Unilever profit rises despite sales slump China',
 'Tesco leads leap FTSE 100 Marks Spencer drops',
 'Royal Dutch Shell profit rises dividend',
 'Morning Agenda Shires Deal NPS',
 'Carnival Corporation China Merchants Group Sign Memo Understanding',
 'Teva FDA Approves Generic Version AstraZeneca Heartburn Drug',
 'Passengers rise EasyJet Aer Lingus',
 'Tesco Abandons VideoStreaming Ambitions Blinkbox Sale',
 'FDA approves NPS drug move validating Shire takeover deal',
 'Ocwen Reaches Settlement With California Regulator',
 'Morrisons faces festive sales test',
 'Tesco share price dips Blinkbox Books closes endin

In [6]:
re.findall(r'\$([a-zA-Z]+)','Slowly adding some $FIO here $googl but gotta be $12 careful. This will be one of biggest winners in 2012')

['FIO', 'googl']

In [72]:
post_sentence = []
post_target = []
extracted_re = []
for key in posts_data.keys():
    post_sentence.append(posts_data[key]['sentence'])
    temp = []
    for x in posts_data[key]['info']:
        temp.append(x['target'])
    post_target.append(temp)
    extracted_re.append(list(set(re.findall(r'\$([a-zA-Z_]+)',posts_data[key]['sentence']))))    

In [73]:
len(post_sentence),len(post_target),len(extracted_re)

(675, 675, 675)

In [20]:
for i in enumerate(zip(target,extracted_re)):
    print(i)

(0, (['FIO'], ['FIO']))
(1, (['TRX'], ['TRX']))
(2, (['AMZN'], ['amzn']))
(3, (['P'], ['P', 'GRPN']))
(4, (['IACI'], ['IACI']))
(5, (['PCLN'], ['pcln']))
(6, (['NFLX'], ['NFLX']))
(7, (['AAPL'], ['AAPL']))
(8, (['SKX'], ['SKX']))
(9, (['MOS'], ['MOS']))
(10, (['TZA'], ['CALL', 'TZA', 'PUT']))
(11, (['YMI'], ['YMI']))
(12, (['TZOO'], ['TZOO']))
(13, (['SWKS'], ['SWKS']))
(14, (['WFT'], ['WFT']))
(15, (['SPPI'], ['SPPI']))
(16, (['SPY'], ['SPY']))
(17, (['IP'], ['IP']))
(18, (['GLD'], ['GLD']))
(19, (['ACOM'], ['ACOM']))
(20, (['ZAGG'], ['ZAGG']))
(21, (['AAPL'], ['AAPL']))
(22, (['LNG'], ['LNG']))
(23, (['MWW'], ['MWW']))
(24, (['SDS'], ['SDS']))
(25, (['IYT'], ['IYT']))
(26, (['DUG'], ['DUG']))
(27, (['SPY'], ['SPY']))
(28, (['AMZN'], ['AMZN']))
(29, (['SLW'], ['SLW']))
(30, (['HSC'], ['HSC']))
(31, (['GMCR'], ['GMCR']))
(32, (['XLF'], ['XLF', 'skf']))
(33, (['VXX'], ['vxx']))
(34, (['GPS'], ['GPS']))
(35, (['WMT'], ['WMT']))
(36, (['MA'], ['MA']))
(37, (['QCOM'], ['GOOG', 'QCOM', 'AMZ

In [21]:
for i,a in enumerate(extracted_re):
    if len(a)>1:
        print(sentence[i])
        print(target[i],a)

$GRPN might be selling off ahead of $P earnings...
['P'] ['P', 'GRPN']
Sold out $TZA 45 $PUT (down $1), which were hedging my 45 $CALL. Letting the calls ride solo now.
['TZA'] ['CALL', 'TZA', 'PUT']
$XLF http://stks.co/1KJ9 Bounced off target buy area, bought $skf as a hedge. watch 50ma's slope
['XLF'] ['XLF', 'skf']
Nasdaq Sets Up for Breakout Part II | via @AllStarCharts http://stks.co/15Nv  $GOOG $AMZN $QCOM
['QCOM'] ['GOOG', 'QCOM', 'AMZN']
$ES_F. Nice little cup-and-handle shaping up on the 15-minute. End-of-day $SPY $SPX favours a good day for the index. Will we see a gap open
['SPY'] ['SPY', 'SPX', 'ES_F']
RT @alphatrends VMarkets Are Still Acting Weak http://stks.co/1agn  $SPY $QQQ $XLF $SMH $IWM
['XLF'] ['XLF', 'IWM', 'QQQ', 'SPY', 'SMH']
Casinos red before the bell: $LVS -0.75%, $WYNN -0.47%, $WYNN -0.47% > unusual comibinations of what's running, what's red before the bell
['WYNN'] ['LVS', 'WYNN']
RT @joemccann the correleation between the dollar index and $SPY is simply am

Looking into Class data first

In [22]:
headlines_data

{'1': {'info': [{'aspects': "['Corporate/Appointment']",
    'sentiment_score': '-0.374',
    'snippets': "['set to step down']",
    'target': 'Royal Mail'}],
  'sentence': 'Royal Mail chairman Donald Brydon set to step down'},
 '100': {'info': [{'aspects': "['Market/Volatility/Volatility']",
    'sentiment_score': '-0.827',
    'snippets': "['down from record high']",
    'target': 'Weir'}],
  'sentence': 'Slump in Weir leads FTSE down from record high'},
 '1000': {'info': [{'aspects': "['Corporate/Regulatory']",
    'sentiment_score': '0.549',
    'snippets': "['FDA approval for key new lung cancer pill']",
    'target': 'AstraZeneca'}],
  'sentence': 'AstraZeneca wins FDA approval for key new lung cancer pill'},
 '1002': {'info': [{'aspects': "['Corporate/Stategy']",
    'sentiment_score': '-0.266',
    'snippets': "['cut 945 jobs as part of 3-year restructuring plan']",
    'target': 'Lloyds'}],
  'sentence': 'UPDATE 1-Lloyds to cut 945 jobs as part of 3-year restructuring plan'},

In [57]:
NNP_tokens=[]
headlines=[]
targets_headline = []
for key in headlines_data.keys():
    text = nltk.word_tokenize(headlines_data[key]['sentence'])
    print("-------------------------------------------------------")
    print(headlines_data[key]['sentence'])
    headlines.append(headlines_data[key]['sentence'])
    temp = []
    for x in headlines_data[key]['info']:
        temp.append(x['target'])
    print(temp)
    targets_headline.append(temp)
    pos_tags = nltk.pos_tag(text)
    for x in pos_tags:
        if x[1] == 'NNP':
            NNP_tokens.append(x[0])
    print(pos_tags)

-------------------------------------------------------
Royal Mail chairman Donald Brydon set to step down
['Royal Mail']
[('Royal', 'NNP'), ('Mail', 'NNP'), ('chairman', 'NN'), ('Donald', 'NNP'), ('Brydon', 'NNP'), ('set', 'VBD'), ('to', 'TO'), ('step', 'VB'), ('down', 'RP')]
-------------------------------------------------------
Stakes High for AstraZeneca Heart Drug Facing Tough Competition
['AstraZeneca']
[('Stakes', 'NNS'), ('High', 'NNP'), ('for', 'IN'), ('AstraZeneca', 'NNP'), ('Heart', 'NNP'), ('Drug', 'NNP'), ('Facing', 'NNP'), ('Tough', 'NNP'), ('Competition', 'NN')]
-------------------------------------------------------
UPDATE 1-Dairy Crest loses a third of Morrisons milk contract
['Morrisons']
[('UPDATE', 'IN'), ('1-Dairy', 'JJ'), ('Crest', 'NNP'), ('loses', 'VBZ'), ('a', 'DT'), ('third', 'JJ'), ('of', 'IN'), ('Morrisons', 'NNP'), ('milk', 'NN'), ('contract', 'NN')]
-------------------------------------------------------
Insight hires Aviva's David Hillier for multi-asset

-------------------------------------------------------
Imperial Tobacco Adds Quarterly Dividends After Profit Rose
['Imperial Tobacco']
[('Imperial', 'JJ'), ('Tobacco', 'NNP'), ('Adds', 'NNP'), ('Quarterly', 'NNP'), ('Dividends', 'NNP'), ('After', 'IN'), ('Profit', 'NN'), ('Rose', 'NN')]
-------------------------------------------------------
BG Group Still Happy With Shell's $70 Billion Offer
['Shell']
[('BG', 'NNP'), ('Group', 'NNP'), ('Still', 'RB'), ('Happy', 'NNP'), ('With', 'IN'), ('Shell', 'NNP'), ("'s", 'POS'), ('$', '$'), ('70', 'CD'), ('Billion', 'NNP'), ('Offer', 'NN')]
-------------------------------------------------------
SABMiller buys Meantime to quench thirst for craft beer
['SABMiller']
[('SABMiller', 'NNP'), ('buys', 'VBZ'), ('Meantime', 'NNP'), ('to', 'TO'), ('quench', 'VB'), ('thirst', 'RB'), ('for', 'IN'), ('craft', 'NN'), ('beer', 'NN')]
-------------------------------------------------------
Aviva shuts Friends Life head office in rapid integration move
['Aviva

['SABMiller']
[('UPDATE', 'IN'), ('2-Pricey', 'JJ'), ('beers', 'NNS'), ('lift', 'VBP'), ('SABMiller', 'NNP'), ("'s", 'POS'), ('quarterly', 'JJ'), ('underlying', 'JJ'), ('sales', 'NNS')]
-------------------------------------------------------
Valeant CEO Pledges to Heed Critics After `Painful' Experience
['Valeant']
[('Valeant', 'NNP'), ('CEO', 'NNP'), ('Pledges', 'NNP'), ('to', 'TO'), ('Heed', 'NNP'), ('Critics', 'NNS'), ('After', 'IN'), ('`Painful', 'JJ'), ("'", 'POS'), ('Experience', 'NN')]
-------------------------------------------------------
Are Aviva plc, Direct Line Insurance Group PLC And Admiral Group plc Set To Soar?
['Aviva', 'Direct Line Insurance Group PLC']
[('Are', 'NNP'), ('Aviva', 'NNP'), ('plc', 'NN'), (',', ','), ('Direct', 'NNP'), ('Line', 'NNP'), ('Insurance', 'NNP'), ('Group', 'NNP'), ('PLC', 'NNP'), ('And', 'CC'), ('Admiral', 'NNP'), ('Group', 'NNP'), ('plc', 'VBZ'), ('Set', 'NNP'), ('To', 'TO'), ('Soar', 'VB'), ('?', '.')]
--------------------------------------

In [58]:
#generating word freq  
# nnp_tokens_freq = nltk.probability.FreqDist(NNP_tokens)
# nnp_tokens_freq.pprint(len(NNP_tokens))

In [33]:
#expermenting with NER

In [59]:
# headlines = [headlines_data[key]['sentence'] for key in headlines_data.keys()]
# targets = 

In [81]:
ners_headlines = [get_ners(s) for s in headlines]
ners_headlines = [(nltk.pos_tag(n)) for n in ners_headlines]
ners_headlines

[[('Donald Brydon', 'NNP'), ('Royal Mail', 'NNP')],
 [('AstraZeneca Heart Drug', 'NN')],
 [('Crest', 'NNP'), ('Morrisons', 'NNS')],
 [('Aviva', 'NNP'), ('Insight', 'NNP'), ('David Hillier', 'NNP')],
 [('Primark', 'NN')],
 [],
 [('Tesco', 'NNP'), ('Blinkbox', 'NNP')],
 [('China', 'NNP'), ('Unilever', 'NNP')],
 [('Tesco', 'NN')],
 [('Royal Dutch Shell', 'NNP')],
 [('Agenda', 'NNP'), ('NPS', 'NNP'), ('Shire', 'NNP')],
 [('China Merchants Group Sign Memo', 'NNP'), ('Carnival Corporation', 'NNP')],
 [('FDA Approves Generic Version', 'NNP'),
  ('Teva', 'NNP'),
  ('AstraZeneca Heartburn Drug', 'NNP')],
 [('Aer Lingus', 'NNP'), ('EasyJet', 'NNP')],
 [('Tesco Abandons', 'NNS'), ('Blinkbox Sale', 'NNP')],
 [('FDA', 'NNP'), ('NPS', 'NNP')],
 [('Ocwen Reaches Settlement', 'NNP'), ('California', 'NNP')],
 [],
 [('Blinkbox Books', 'NNP'), ('Tesco', 'NNP')],
 [('NPS', 'NNP'), ('Shire', 'NNP')],
 [('MedImmune Inks', 'NNS'), ('AstraZeneca', 'NNP'), ('Omnis', 'NNP')],
 [('FDA', 'NNP'), ('Shire', 'NNP')]

In [83]:
for i in enumerate(zip(targets_headline,ners_headlines)):
    print(i)

(0, (['Royal Mail'], [('Donald Brydon', 'NNP'), ('Royal Mail', 'NNP')]))
(1, (['AstraZeneca'], [('AstraZeneca Heart Drug', 'NN')]))
(2, (['Morrisons'], [('Crest', 'NNP'), ('Morrisons', 'NNS')]))
(3, (['Insight'], [('Aviva', 'NNP'), ('Insight', 'NNP'), ('David Hillier', 'NNP')]))
(4, (['Primark'], [('Primark', 'NN')]))
(5, (['Pearson'], []))
(6, (['TalkTalk', 'Tesco'], [('Tesco', 'NNP'), ('Blinkbox', 'NNP')]))
(7, (['Unilever'], [('China', 'NNP'), ('Unilever', 'NNP')]))
(8, (['Tesco'], [('Tesco', 'NN')]))
(9, (['Royal Dutch Shell'], [('Royal Dutch Shell', 'NNP')]))
(10, (['Shire'], [('Agenda', 'NNP'), ('NPS', 'NNP'), ('Shire', 'NNP')]))
(11, (['China Merchants Group'], [('China Merchants Group Sign Memo', 'NNP'), ('Carnival Corporation', 'NNP')]))
(12, (['AstraZeneca', 'Teva'], [('FDA Approves Generic Version', 'NNP'), ('Teva', 'NNP'), ('AstraZeneca Heartburn Drug', 'NNP')]))
(13, (['EasyJet'], [('Aer Lingus', 'NNP'), ('EasyJet', 'NNP')]))
(14, (['Blinkbox', 'Tesco'], [('Tesco Abandons'

In [68]:
headlines[284]

'Barclays sells benchmark indices unit to Bloomberg'

In [69]:
posts_data

{'14860': {'info': [{'aspects': "['Stock/Price Action/Bullish/Bull Position']",
    'sentiment_score': '0.459',
    'snippets': "['Slowly adding some $FIO here but gotta be careful']",
    'target': 'FIO'}],
  'sentence': 'Slowly adding some $FIO here but gotta be careful. This will be one of biggest winners in 2012'},
 '14864': {'info': [{'aspects': "['Stock/Technical Analysis']",
    'sentiment_score': '0.438',
    'snippets': "['Long setup. MACD cross.']",
    'target': 'TRX'}],
  'sentence': '$TRX http://stks.co/1KkK Long setup. MACD cross.'},
 '14867': {'info': [{'aspects': "['Stock/Price Action/Bearish']",
    'sentiment_score': '-0.506',
    'snippets': "['both fundementals and charts look like poopoo this quarter.']",
    'target': 'AMZN'}],
  'sentence': 'I am not optimistic about $amzn both fundementals and charts look like poopoo this quarter.'},
 '14875': {'info': [{'aspects': "['Stock/Price Action/Bearish/Bearish behaviour']",
    'sentiment_score': '-0.202',
    'snippets

In [75]:
post_target

[['FIO'],
 ['TRX'],
 ['AMZN'],
 ['P'],
 ['IACI'],
 ['PCLN'],
 ['NFLX'],
 ['AAPL'],
 ['SKX'],
 ['MOS'],
 ['TZA'],
 ['YMI'],
 ['TZOO'],
 ['SWKS'],
 ['WFT'],
 ['SPPI'],
 ['SPY'],
 ['IP'],
 ['GLD'],
 ['ACOM'],
 ['ZAGG'],
 ['AAPL'],
 ['LNG'],
 ['MWW'],
 ['SDS'],
 ['IYT'],
 ['DUG'],
 ['SPY'],
 ['AMZN'],
 ['SLW'],
 ['HSC'],
 ['GMCR'],
 ['XLF'],
 ['VXX'],
 ['GPS'],
 ['WMT'],
 ['MA'],
 ['QCOM'],
 ['ISRG'],
 ['UTSI'],
 ['GMCR'],
 ['AAPL'],
 ['AAPL'],
 ['DPZ'],
 ['SPY'],
 ['FAZ'],
 ['XLF'],
 ['BRCM'],
 ['ELN'],
 ['WYNN'],
 ['TSPT'],
 ['SKH'],
 ['LULU'],
 ['SPY'],
 ['FIO'],
 ['AAPL'],
 ['ROST'],
 ['DIA'],
 ['LULU'],
 ['ZSL'],
 ['FMCN'],
 ['MCP'],
 ['ACE'],
 ['AAPL'],
 ['CIEN'],
 ['T'],
 ['DWA'],
 ['IBM'],
 ['SINA'],
 ['AMZN'],
 ['SIGA'],
 ['YHOO'],
 ['CERN'],
 ['SPY'],
 ['RDC'],
 ['NVDA'],
 ['HITK'],
 ['WMT'],
 ['GOOG'],
 ['AAPL'],
 ['FIO'],
 ['SPY'],
 ['LNT'],
 ['UUP'],
 ['IDCC'],
 ['MAKO'],
 ['SPY'],
 ['TZA'],
 ['SSRI'],
 ['SPY'],
 ['AAPL'],
 ['VVUS'],
 ['CREE'],
 ['YELP'],
 ['RENN'],
 ['SPY'],


In [76]:
ners_post = [get_ners(s) for s in post_sentence]

In [77]:
for i in enumerate(zip(post_target,ners_post)):
    print(i)

(0, (['FIO'], []))
(1, (['TRX'], ['MACD', 'Long']))
(2, (['AMZN'], []))
(3, (['P'], ['GRPN']))
(4, (['IACI'], ['IACI', 'Looks']))
(5, (['PCLN'], []))
(6, (['NFLX'], ['RT']))
(7, (['AAPL'], []))
(8, (['SKX'], ['Stock']))
(9, (['MOS'], []))
(10, (['TZA'], ['CALL']))
(11, (['YMI'], ['MACD', 'Long']))
(12, (['TZOO'], []))
(13, (['SWKS'], ['SWKS']))
(14, (['WFT'], []))
(15, (['SPPI'], ['Sweet']))
(16, (['SPY'], []))
(17, (['IP'], ['Double']))
(18, (['GLD'], ['RSI']))
(19, (['ACOM'], ['ACOM']))
(20, (['ZAGG'], ['Volume']))
(21, (['AAPL'], []))
(22, (['LNG'], ['Someone']))
(23, (['MWW'], ['Covered', 'Flat']))
(24, (['SDS'], []))
(25, (['IYT'], ['MACD', 'RT', 'Transports']))
(26, (['DUG'], ['Buying']))
(27, (['SPY'], ['QE3']))
(28, (['AMZN'], []))
(29, (['SLW'], ['SLW']))
(30, (['HSC'], ['HSC Receives', 'New Saudi Arabian Steel Mill']))
(31, (['GMCR'], ['GMCR']))
(32, (['XLF'], []))
(33, (['VXX'], []))
(34, (['GPS'], ['GPS Gap']))
(35, (['WMT'], ['Double']))
(36, (['MA'], []))
(37, (['QCOM'], 